# Spark RDDs
This notebook is intended to teach you the basic structure of Apache Spark, from the basic structures to some of the more advanced techniques.  We'll do our best to give you practical advice, and not get bogged down in little minutia.

## Initializing Spark
We must begin by making sure that the needed JARs are present on the system, and that the interpreter knows we're loading them into the classpath.

In [ ]:
import $ivy.`org.apache.logging.log4j:log4j-core:2.17.0`
import $ivy.`org.apache.logging.log4j:log4j-1.2-api:2.17.0`
import $ivy.`org.apache.spark::spark-sql:3.3.1`

As a convenience, let's also quiet the logging facility.

In [ ]:
import org.apache.log4j.{Level, Logger}
Logger.getRootLogger.setLevel(Level.WARN)
Logger.getLogger("org").setLevel(Level.OFF)
Logger.getLogger("org").setLevel(Level.OFF)

Next, we can start a `SparkSession`; this object manages the interaction between the Spark execution engine and this interpreter.

In [ ]:
import org.apache.spark.sql._

val spark = {
  NotebookSparkSession.builder()
    .master("local[4]")
    .getOrCreate()
}

Because we're going to begin our introduction to Spark by using the older RDD interface, we're going to make available the `SparkContext` that is wrapped by `spark`.  We're going to declare this as an `implicit val` because some functions use an implicit argument to access it to keep us from needing to pass the value explicitly.

In [ ]:
implicit val sc = spark.sparkContext

## Resilient Distributed Datasets (RDDs)
The heart and soul of Spark is the RDD.  This is an immutable, distributed data structure, where the data are spread across Spark's worker nodes (also called _executors_).  It is also fault tolerant (hence the R in RDD), in the fact that Spark maintains a call graph for each partition, so if an executor is lost, the partition can be shifted to another node and recalculated.

The RDD is the base level structure in Spark, and provides a host of methods for manipulating the contained data.  Most of these functions take on a functional flavor, particularly due to the immutability of the RDD.  Therefore, our code will appear to use a sequence of _transformations_ of a base RDD (often loaded from a remote data store) to generate the result we need.

### Creating RDDs from scratch

If you have programmatically-generated data that is small enough to fit entirely in driver memory, you can use the `parallelize` method of the `SparkContext`.  Note that this will not work for very large data.

In [ ]:
val sequenceRDD = sc.parallelize(Range(1, 10000).toSeq)

There are also built-in methods for loading text from local files (`textFile`) or loading files from the Hadoop file system (`hadoopFile`).  It will be up to the user to convert these inputs into something useful.

Given an RDD, it is then our job to construct a collection of [transformations](https://spark.apache.org/docs/latest/rdd-programming-guide.html#transformations) that convert the input data into a useful output, which could be a single value or a sequence.  This chain of transformations constructs an internal call graph in the Spark execution engine which is lazily computed.  Only as much data as are needed will be pulled in to achieve the required result.  There are a smaller number of [actions](https://spark.apache.org/docs/latest/rdd-programming-guide.html#actions) which actually trigger computations.

As an example, the above RDD can be filtered to hold just the values below a threshold:

In [ ]:
val filtered = sequenceRDD.filter{ x => x < 5000 }

Here, we only _describe_ a computation.  To do something with it, we have to call an action.  We can try a reduce operation to find the sum of the elements:

In [ ]:
filtered.reduce(_ + _)

We could have made a more complex chain of operations prior to this reduce step to generate a more interesting result:

In [ ]:
import scala.math.log
val filteredAndMapped = filtered.map{ x => log(x) }

In [ ]:
filteredAndMapped.reduce(_ + _)

Note that nothing was stopping us from mapping and then filtering:

In [ ]:
sequenceRDD.map{ x => log(x) }.filter(_ < log(5000)).reduce(_ + _)

Thus, users of the RDD Spark interface must take care to choose an optimal computation path, or excess computation will occur.

## A Geotrellis example

It's fairly limiting to work with synthetic data and contrived mappings, so we're going to look at a more complicated example that is only 

### Loading RDDs from a file

Let's load a dataset that we can mess around with.  The dataset we've chosen is an extract of parcel information from the city of Philadelphia.  The data are formatted in line-delimited GeoJSON, so each line is a GeoJSON feature containing a JTS Polygon with a map of additional information.  There are not a wide array of options for loading data into RDDs, but in this case, we can use the `textFile()` method on the `SparkContext`, which creates an RDD where each element is a line from the source text file.

In [ ]:
val parcel_raw = sc.textFile("data/extract.geojson.ld")

At the moment, we have an `RDD[String]`, which is not very useful.  For this demonstration, we'd like to pull only the geometries from these records.  This requires some new imports:

In [ ]:
import $ivy.`org.locationtech.geotrellis::geotrellis-vector:3.6.3`

import geotrellis.vector._
import geotrellis.vector.io._
import geotrellis.vector.io.json._

These imports bring some elements of Circe, the Scala JSON parsing library, into scope, so now we can see what we're working with:

In [ ]:
println(parcel_raw.first.parseJson)

A note that we used the RDD's `first` method which triggered a simple computation.  We can observe the tracking information provided by Spark at https://localhost:4040 (if you didn't redirect to a different port when you started the container).

### Laziness and Persistence

The above string can be parsed and converted to a Polygon using the following function:

In [ ]:
def convertToPolygon(s: String): Polygon = {
    val parsed = s.parseJson
    parsed.hcursor.downField("geometry").as[Polygon].getOrElse(GeomFactory.factory.createPolygon())
}

In [ ]:
val parcels = parcel_raw.map(convertToPolygon)

This application of `map` will not trigger any work, due to Spark's laziness, which we discussed earlier.  Confirm this by looking at the Spark UI; there should be no new jobs listed.  The `parcels` RDD is presently a description of work to be done.  Once a triggering action is called, this object becomes realized, but, Spark is not guaranteed to hold onto the contents of `parcels`.  This is not a problem unless we intend to use `parcels` as the base of more than one computation.  If we do, we might trigger multiple recomputations of the same RDD.  This can be prevented by calling `cache()` or `persist()` on the RDD of interest.

In [ ]:
parcels.persist(org.apache.spark.storage.StorageLevel.MEMORY_AND_DISK)
// parcels.cache()   // Equivalent to persisting to StorageLevel.MEMORY_ONLY

Persisting an RDD will add it to the cluster's storage, and there are a variety of [storage modes](https://spark.apache.org/docs/3.2.0/api/java/org/apache/spark/storage/StorageLevel.html) to choose from.  We must take care not to overuse this mechanism, lest we run out of space on our nodes, but it can significantly speed up some workflows.

After calling persist on this RDD, if we go to the Storage tab in the Spark UI, we'll see nothing, because we still have not executed the code to build the lazy RDD.  But if we do something with `parcels`, it should appear.  So, let's trigger a fairly innocuous action of grabbing the first element of the RDD:

In [ ]:
parcels.first

Refreshing the Storage tab should show that we did some work and stashed it for later, but because the work we did only needed a single element, we didn't compute the whole dataset.

We can compute an aggregate area for all the parcels, which will use the whole dataset, caching the remaining blocks:

In [ ]:
parcels.map(_.reproject(geotrellis.proj4.LatLng, geotrellis.proj4.WebMercator).getArea).fold(0.0)(_ + _)

With that done, we can unpersist the parcels, since we don't need it stored any longer.

In [ ]:
parcels.unpersist(true) // The true says to block until the storage is removed

### Paired RDD Operations

Many interesting operations with RDDs require that we key our data.  Once keyed, we can group records to compute aggregated values.  This would be the way to figure out, for example, how much area in Philadelphia is devoted to which zoning type (if we had zoning information for each parcel): key by zone type and aggregate by summing areas per zone.  In our case, we're going to create a mask raster for our sample of parcels by

1. keying each parcel to a grid cell for some layout,
2. rasterizing each parcel to a raster chip,
3. merging all the parcel rasters for a given grid cell, and
4. stitching the results into a final raster.

In actual practice, steps 2 and 3 will be combined, so this won't be as inefficient as it sounds.  The total size of the computation will be dominated by the number of grid cells and the resolution of each chip.

We're going to use some additional Geotrellis facilities for this, so let's import them:

In [ ]:
import $ivy.`org.locationtech.geotrellis::geotrellis-raster:3.6.3`
import $ivy.`org.locationtech.geotrellis::geotrellis-layer:3.6.3`

import geotrellis.raster._
import geotrellis.layer._

To key each parcel to a grid, we're going to need to establish a layout.  We'll use the Geotrellis layout scheme that corresponds to the power-of-two pyramid often used for web maps.  The grid that we'll use for this exercise will be defined at a fixed zoom level.  The finer the zoom, the more cells, and therefore, keys we'll have to work with, but also the bigger the final image that we'll produce.

In [ ]:
val zoom = 14
val LayoutLevel(_, layout) = ZoomedLayoutScheme(geotrellis.proj4.WebMercator).levelForZoom(zoom)

Let's get some idea of the scale of the problem that we're going to be looking at.  For starters, how many keys will we be working with?

To make these calculations, it will be important to make sure that our parcel boundaries are in the correct projection.  Then, we can figure out the extent of the whole dataset and determine how many grid cells are going to participate.

In [ ]:
val wmParcels = parcels.map(_.reproject(geotrellis.proj4.LatLng, geotrellis.proj4.WebMercator))
val totalExtent = wmParcels.map(_.extent).reduce(_.combine(_))

In [ ]:
val mapTransform = layout.mapTransform
val regionBounds = mapTransform.extentToBounds(totalExtent)
// The following will compute the upper bound of the number of grid cells
// Not every cell is guaranteed to have a member
println(f"Upper bound on the count of cells: ${regionBounds.coordsIter.length} (grid region dimensions: ${(regionBounds.width, regionBounds.height)})")

Given this precalculation, we can infer that, as long as the individual chips that we use in our layout aren't too big, we can make a reasonably-sized final image.  

Let's proceed with assigning a key to each parcel.  Note, however, that some parcels will cross grid cell boundaries, so we have to produce a sequence of keys for each parcel and merge all the results.  This is the role of a `flatMap`.

In [ ]:
val keyedParcels = wmParcels.flatMap{ parcel =>
    mapTransform.keysForGeometry(parcel).map{ k => (k, (k, parcel)) }
}

For reasons that will be clear below, we need to replicate the grid cell location in both the key and value portion of the paired RDD.

### Partitions and Shuffling

Spark distributes data across a set of worker nodes, called _executors_.  An RDD is comprised of _partitions_, with the partitions being spread across the cluster's executors.  There is no requirement for the partitions to be of equal size, so _partition skew_ is more than possible, and this means that some executors will have more work to do than others because the data are distributed heterogeneously, and some unevenness is likely.

We can do things to make this problem better or worse.  If the problem set has, like ours, an inherently non-uniform distribution of data (some geographical areas are more densely populated), then grouping by key is likely to produce skewed partitions.  So, even though this is the logical first step, it's better to just work on the data where they are, and pass smaller, intermediate results between executors as we combine.

For this problem, the intermediate data are the small raster chips that we're going to rasterize our footprints to.  These will have a fixed and relatively small size, while the number of raw data elements to be shuffled in a group by key could be much, much larger.

Considering the ways to make data distributions better, choosing a good number of partitions is one thing we can do.  Let's see how many we have now:

In [ ]:
println(keyedParcels.partitions.length)

Depending on the number of workers, this may be a good or bad number of partitions.  We might target some number of partitions per executor, but there is also a management penalty for having too many, which is borne by the master node as additional memory and processing overhead.  We'll increase the number of partitions here just to show how it is done:

In [ ]:
val keyedParcelsRP = keyedParcels.repartition(keyedParcels.partitions.length * 4)
println(keyedParcelsRP.partitions.length)

### Completing the exercise

Now, we can go about doing the conversion from parcel geometries to mask rasters.  The approach will be to use [`combineByKey`](https://spark.apache.org/docs/0.7.3/api/core/spark/PairRDDFunctions.html), which essentially folds per-key, per-partition, shuffles the intermediate results, and then merges them to get the final result per key.  This requires defining an initial value, explaining how to add a value to it, and providing a means to merge the intermediates.

In [ ]:
import geotrellis.raster.rasterize._

// Define the chip dimensions
val chipSize = 128

def burnParcel(parcel: Polygon, key: SpatialKey, tile: BitArrayTile): Unit = {
    val rasterExtent = RasterExtent(mapTransform.keyToExtent(key), chipSize, chipSize)
    parcel.foreach(rasterExtent){ (r: Int, c:Int) => tile.set(r, c, 1) }
}

def createFromParcel(keyedParcel: (SpatialKey, Polygon)): BitArrayTile = {
    val tile = BitArrayTile.empty(chipSize, chipSize)
    val (key, parcel) = keyedParcel
    burnParcel(parcel, key, tile)
    tile
}

def addParcelToTile(tile: BitArrayTile, keyedParcel: (SpatialKey, Polygon)): BitArrayTile = {
    val (key, parcel) = keyedParcel
    burnParcel(parcel, key, tile)
    tile
}

def mergeTiles(tile1: BitArrayTile, tile2: BitArrayTile): BitArrayTile =
    tile1.combine(tile2){ (v1, v2) => if (v1 + v2 > 0) 1 else 0 }.asInstanceOf[BitArrayTile]

val tileRDD = keyedParcelsRP.combineByKey(createFromParcel, addParcelToTile, mergeTiles)

At this point, we have a tile per non-empty SpatialKey.  We need to assemble the result into a complete raster.  There are a number of ways one could conceive of doing this, but fortunately, Geotrellis already offers [stitching](https://github.com/locationtech/geotrellis/blob/master/spark/src/main/scala/geotrellis/spark/stitch/StitchRDDMethods.scala) extension methods.  The only thing we need to do is satisfy the base type requirement of `RDD[(SpatialKey, T)] with Metadata[M]`, where `V` is a compatible tile type and `M` carries a layout definition.  For this reason, Geotrellis provides the `ContextRDD` and `TileLayerMetadata` classes.

In [ ]:
import $ivy.`org.locationtech.geotrellis::geotrellis-spark:3.6.3`
import geotrellis.raster.render._
import geotrellis.spark._
import geotrellis.spark.stitch._

In [ ]:
val metadata = TileLayerMetadata(
    BitCellType, 
    layout, 
    mapTransform.boundsToExtent(regionBounds), 
    geotrellis.proj4.WebMercator, 
    KeyBounds(regionBounds)
)
val tilesWithMetadata = ContextRDD(tileRDD, metadata).asInstanceOf[ContextRDD[SpatialKey, Tile, TileLayerMetadata[SpatialKey]]]

In [ ]:
tilesWithMetadata
  .stitch
  .tile
  .renderPng(ColorMap(Map(0 -> RGB(0,0,0), 1->RGB(255,255,0))))
  .write("test.png")

### A note about shuffling and serialization

It's the case that when Spark sends objects over the wire from executor to another executor or the driver, these objects need to be converted into a byte stream.  That stream needs to be generated somehow, but how that happens matters for performance.  Spark can use either plain old Java serialization or it can use Kryo serialization.  The former is more ubiquitous, the latter more performant.  In order to use the latter, one must configure the spark session to use it.  There is a Spark configuration field `spark.kryo.registrator` that can be set to `classOf[KryoRegistrator].getName`, but in that case, it is best to also set `spark.kryo.registrationRequired` to false, to avoid serialization errors.  Serialization errors are often not generated when using a local master, so testing new code on a genuine, multi-node cluster is advisible before deploying code.

## Exercise

1. Generate some random `(x, y)` pairs of Doubles over the unit square.
2. Grid up the unit square into a number of cells, and assign each point a key based on the cell it falls inside.
3. Compute the average of the points in each cell.
4. Compute the minimum, maximum, and average distance between the cell centers and the per-cell average

Parameterize this process, and see how the deviations change as the number of points increases and the number of grid cells increases.

## Clean up

To ensure that we aren't holding the Spark UI port, run the following cell:

In [ ]:
spark.stop